In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# add root to sys path
import os
import sys

current_dir = os.getcwd()
root = os.path.abspath(os.path.join(current_dir,".."))
if root not in sys.path:
    sys.path.append(root)

In [ ]:
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = ANTHROPIC_API_KEY

# Imports

import agent
from langchain_core.messages import AIMessage, HumanMessage

graph = agent.graph

orchestrator = agent.orchestrator
run_control_flow = agent.run_control_flow
generate_answer = agent.generate_answer
create_sql_query_or_queries = agent.create_sql_query_or_queries
extract_analytical_intent = agent.extract_analytical_intent

# Import initialization components
from src.init.initialization import (
    llm, llm_fast, create_config, tracer,
    objects_documentation, sql_dialect, connection_string
)

question = 'placeholder'
test_state = {
'objects_documentation':objects_documentation,
'sql_dialect': sql_dialect,
'messages_log':[],
'intermediate_steps' : [],
'analytical_intent': [],
'current_question':question,
'current_sql_queries': [],
'generate_answer_details': {
    'key_assumptions': [],
    'agent_questions': [],
    'ambiguity_explanation': ''
},
'llm_answer': AIMessage(content=''),
'scenario': ''
,'search_terms_output':[]
}

In [ ]:
# start the conversation with the graph
question = 'aum by practice segments'

test_state['current_question'] = question
vector_store = None  # reset vector store
config, thread_id = create_config('Run Agent',True)

result = graph.invoke(test_state, config = config)
display = f'''Analytical intent: {result['analytical_intent']}\n\nSQL query: {result['current_sql_queries']}\n\nGenerate Answer Details: {result['generate_answer_details']}\n\nAnswer: {result['llm_answer'].content}'''
print(display)

In [ ]:
# continue the conversation with the graph (followup 1)
question = 'can you say again which one is the best one? sorry i missed it'

test_state['current_question'] = question
vector_store = None  # reset vector store
config, _ = create_config('Run Agent', False, thread_id) # (re-use same thread)
result = graph.invoke(test_state, config)
display = f'''Analytical intent: {result['analytical_intent']}\n\nSQL query: {result['current_sql_queries']}\n\nGenerate Answer Details: {result['generate_answer_details']}\n\nAnswer: {result['llm_answer']}'''
print(display)

In [ ]:

user_question = 'For Oak Wealth, what was their EOM asset value and payout?'
followup_question = 'How much of that was mutual fund data or cash equivalent?'

user_question = 'What is distinct count of active advisors?'

user_question = 'List all advisors affiliated with firm Crescent Wealth LLC.'

user_question = 'Group advisors by tenure group. Compare average total assets, net revenue across these groups. Which group shows the highest revenue per advisor?'

# User asks for synonyms
user_question = 'aum by practice segments'
# expecting to query for assets under management and to not aggregate it over time.

# User asks for a vague term (undefined at org level) but related to multiple terms available in db 
user_question = 'Payments associated with advisors from firm Cedar Capital LLC'
# expecting to enter disambiguation node and say that payment can mean net revenue (Revenue retained by Capital Partners) or payout (Dollar amount paid to advisor), and ask the user which one it prefers.

# User asking with lazy language: 
user_question = 'net revenue associated with Cedar Capital advisors'
# expecting to filter for sum of net revenue for advisors belonging to the firm Cedar Capital LLC

# User asks for a term not in db (but defined) but related to multiple terms available in db 
user_question = 'distribution of advisor ID 8'
# expecting to enter disambiguation node and say that distribution (advisor payout after tech fees are deducted) is not in tables it has access to, 
# but it can offer net revenue (Revenue retained by Capital Partners) or payout (Dollar amount paid to advisor), which one you prefer?

# User asks for a vague term (undefined at org level) but related to a single terms available in db: producing assets (vague): 
user_question = 'producing assets by practice segments'
# expecting to say to query for advisory assets and say that advisory assets is Assets in Managed Portfolio and SMA business lines.

# User asks for a term not in db (but defined) but related to a single terms available in db
user_question = 'liquid assets by practice segments'
# expecting to say it doesn't have access to liquid assets (assets easily converted to cash) but here are the results for advisory assets (Assets in Managed Portfolio and SMA business lines).
# filter for account_type = 'Custody', account.account_status = 'Active' and account.to_date = '9999-12-31'